## Querying a Milvus index

Simple example on how to query content from a Milvus VectorStore.

Requirements:
- A Milvus instance, either standalone or cluster.
- Connection credentials to Milvus must be available as environment variables: MILVUS_USERNAME and MILVUS_PASSWORD

### Needed packages and imports

In [1]:
!pip install -q einops==0.7.0 langchain==0.1.9 pymilvus==2.3.6 sentence-transformers==2.4.0


[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Milvus

### Base parameters, the Milvus connection info

In [3]:
# Base parameters for Milvus connection
MILVUS_HOST = "vectordb-milvus.milvus.svc.cluster.local"
MILVUS_PORT = 19530
MILVUS_USERNAME = "root"
MILVUS_PASSWORD = "Milvus"
MILVUS_COLLECTION = "catalogo_ba_gov"

### Initialize the connection

In [43]:
# If you don't want to use a GPU, you can remove the 'device': 'cuda' argument
model_kwargs = {'trust_remote_code': True}
embeddings = HuggingFaceEmbeddings(
    model_kwargs=model_kwargs,
    show_progress=True
)

store = Milvus(
    embedding_function=embeddings,
    connection_args={"host": MILVUS_HOST, "port": MILVUS_PORT, "user": MILVUS_USERNAME, "password": MILVUS_PASSWORD},
    collection_name=MILVUS_COLLECTION,
    metadata_field="metadata",
    text_field="page_content",
    drop_old=False
    )

### Make a query to the index to verify sources

In [48]:
query="Quais são os documentos necessários para renovação do IPTU do estado de Sao Paulo "
results = store.similarity_search_with_score(query, k=2, return_metadata=True)
for result in results:
    print(result[0].metadata['source'])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

markdown/IPTU.md
markdown/IPTU.md


### Work with a retriever

In [49]:
retriever = store.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [50]:
docs = retriever.get_relevant_documents(query)
docs

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[Document(page_content='### DESCRIÇÃO DO SERVIÇO  \nServiço destinado aos cidadãos que desejam obter a Emissão de 2º Via de IPTU do município de Lauro de Freitas.  \n  \nO IPTU é um imposto que incide sobre a propriedade, domínio útil ou posse imobiliária urbana, tanto para pessoas físicas, como para jurídicas. Com alíquota definida em lei municipal e benefício de desconto, definido no calendário fiscal.  \n  \n### PRÉ-REQUISITO DO SERVIÇO', metadata={'source': 'markdown/IPTU.md'}),
 Document(page_content='### ÓRGÃO GESTOR  \nSecretaria da Fazenda do Município de Lauro de Freitas - SEFAZ  \n  \n### INFORME  \nPara solicitar o serviço proceda da seguinte forma:  \n01. Clique em "solicitar";  \n02. Informe o “número de inscrição do imóvel” e o “exercício” e clique em "avançar";  \n03. Informe o “e-mail” para receber o boleto para pagamento (opcional) e clique em "próximo";  \n04. Clique em “emitir 2ª via”.  \n  \n### RELAÇÃO DE DOCUMENTOS  \n01. Número de Inscrição Municipal.  \n  \n### 